# Complex Pulses on Baseband RF Signals

A common use case for HDAWG instruments is to use its baseband outputs individually (i.e. without combining pairs into IQ signals).
It is possbile to use complex-valued pulse envelopes on such signals, however there is a nuance related to their modulation.

If software modulation is used, it fully allows programming complex-valued baseband signals on RF signal lines. LabOneQ modulates the complex-valued pulse envelope with a complex oscillator, as if modulating an IQ signal. The physical output is the real part of the modulation result.

The situation is different when hardware modulation is used - there is only a single hardware oscillator, hence it cannot modulate the signal in its full complex form. Thus, the imaginary part of the pulse envolpe is ignored, and a warning is logged.

In conclusion, in use cases where baseband RF signals are used in conjuction with complex-valued pulse envelopes, you should use software modulation.
In use cases where baseband RF signals are used with real-valued pulse envelopes, both modulation modes can be used at your convenience.

Below, we demonstrate the difference between using different modulation types for RF signals with complex-valued pulse envelopes.

# Simple Example

Let's consider a minimal example with the below setup and a corresponding session object:

In [ ]:
from laboneq.simple import HDAWG, DeviceSetup, Session, create_connection

setup = DeviceSetup("rf_signal_setup")
setup.add_dataserver(host="localhost", port="8004")
setup.add_instruments(HDAWG(uid="hdawg", address="dev8047"))
setup.add_connections(
    "hdawg", create_connection(to_signal="q0/rf_line", ports="SIGOUTS/0")
)

session = Session(device_setup=setup)
_ = session.connect(do_emulation=True)

Now, let's build a simple experiment that plays a few DRAG pulses on the line `q0/rf_line`. DRAG pulses are complex-valued.

In [ ]:
from laboneq.dsl.experiment.builtins import *
from laboneq.simple import ModulationType, Oscillator, SignalCalibration, pulse_library


@experiment(signals=["rf_line"])
def exp(modulation_type: ModulationType):
    freq = 1e6
    drag = pulse_library.drag(length=104e-9, amplitude=0.1)
    with acquire_loop_rt(1):
        for _ in range(20):
            play("rf_line", pulse=drag)

    map_signal("rf_line", setup.logical_signal_groups["q0"].logical_signals["rf_line"])
    experiment_calibration()["rf_line"] = SignalCalibration(
        oscillator=Oscillator(frequency=freq, modulation_type=modulation_type)
    )

First, let us build and compile this experiment using software modulation:

In [ ]:
compiled = session.compile(exp(ModulationType.SOFTWARE))

Below, we can view the output waveforms using the output simulator.
As can be seen the pulse envelope starts as Gaussian, gradually transforms into a Gaussian derivative (as the oscillator accumulates $\pi / 2$ phase), then goes back to Gaussian, and so forth. This is precisely because the pulse underwent complex modulation.
We have intentionally used a very low modulation frequency, so that the complex-valued nature of the modulation is easy to demonstrate.

In [ ]:
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation

plot_simulation(compiled, 0, 2000e-9, plot_height=3)

Now, let's do the same but using hardware modulation:

In [ ]:
compiled = session.compile(exp(ModulationType.HARDWARE))

The output of this one we cannot fully view using the output simulator, since part of the waveform formation (the modulation) happens on hardware.
An oscilloscope is needed to capture the output. Below is a screenshot of the output from an oscilloscope. As can be see, it differs from the software modulation case. 
As the oscillator accumulates phase, we can only see scaled Gaussian envelopes - the derivative component is not present since it was ignored during modulation.

In [ ]:
# _ = session.run(compiled)

<div style="max-width:800px;">

![image.png](images/drag_rf_hw_mod.png)

</div>